In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import leveldb
import lmdb
import time

caffe_root = os.environ['CAFFE_PROJ_DIR']
print 'caffe_root ',caffe_root
sys.path.insert(0, caffe_root + 'python')
sys.path.append(caffe_root + 'python/util')
import caffe
from util_func import *

sys.path.append(caffe_root + 'python/caffe/proto')
import caffe_pb2

# Set the right path to your model definition file, pretrained model weights,
# and the image you would like to classify.

'''building block net ImageNet-NIN'''
# model_dir=caffe_root+'models/nin_imagenet/'
# MODEL_FILE =model_dir + 'deploy.prototxt'
# PRETRAINED = model_dir + 'nin_imagenet_train_iter_0.caffemodel'
# raw_scale=255

'''HDCNN (ImageNet-NIN)'''
model_dir=caffe_root+'models/nin_imagenet/89clusters/89clusters_v2.0/'
MODEL_FILE =model_dir + 'hdcnn_deploy.prototxt'
PRETRAINED = model_dir + 'hdcnn_iter_0.caffemodel'
raw_scale=255

# model_dir=caffe_root+'models/VGG_ILSVRC_16_layers/211839e770f7b538e2d8/'
# MODEL_FILE =model_dir + 'deploy.prototxt'
# PRETRAINED = model_dir + 'VGG_ILSVRC_16_layers.caffemodel'
# raw_scale=255

# model_dir = caffe_root + 'models/bvlc_reference_caffenet/'
# MODEL_FILE = model_dir + 'deploy.prototxt'
# PRETRAINED = model_dir + 'bvlc_reference_caffenet.caffemodel'
# raw_scale=255


example_dir=caffe_root+'examples/imagenet/'
data_dir=caffe_root+'data/ilsvrc12/'
print 'model_dir ',model_dir
print 'example_dir ',example_dir
print 'data_dir ',data_dir

caffe_root  /home/zyan3/proj/caffe_private_hdcnn/
model_dir  /home/zyan3/proj/caffe_private_hdcnn/models/nin_imagenet/89clusters/89clusters_v2.0/
example_dir  /home/zyan3/proj/caffe_private_hdcnn/examples/imagenet/
data_dir  /home/zyan3/proj/caffe_private_hdcnn/data/ilsvrc12/


In [4]:
print MODEL_FILE
print model_dir


/home/zyan3/proj/caffe_private_hdcnn/models/nin_imagenet/89clusters/89clusters_v2.0/hdcnn_deploy.prototxt
/home/zyan3/proj/caffe_private_hdcnn/models/nin_imagenet/89clusters/89clusters_v2.0/


In [3]:
label_names=read_text(data_dir + 'synset_words.txt')
num_class=len(label_names)
print 'num_class: %d' % num_class

num_class: 1000


In [4]:
caffe.set_mode_gpu()
caffe.init_device(0)

In [5]:
# caffe.init_device(1)
# when loading individual images using caffe.io.load_image, channel_swap=(2,1,0)
# when loading images from preprocessed database, channel_swap=(0,1,2)
net = caffe.Classifier(MODEL_FILE, PRETRAINED,
                       mean=np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy'),
                       channel_swap=(0,1,2),
                       raw_scale=raw_scale,
                       image_dims=(256,256))

In [6]:
val_img_list_fn = data_dir + 'val.txt'
val_img_names, val_img_labels=read_val_image_list(val_img_list_fn)
num_val = len(val_img_names)

50000 validation images 


In [7]:
# val_lmdb_fn = '/home/zyan3/local/data/imagenet/ilsvrc12/ilsvrc12_val_size384_lmdb'
val_lmdb_fn = '/home/zyan3/local/data/imagenet/ilsvrc12/ilsvrc12_val_size_256_lmdb'
val_lmdb = lmdb.open(val_lmdb_fn)
txn=val_lmdb.begin()

In [13]:
prediction_single_sum=np.sum(prediction_single,axis=1)
print prediction_single_sum.shape
print prediction_single_sum[:10]
'''need L1 normalization'''

(50000,)
[ 10.54082394  13.047719    10.93774033  10.80950356  12.81902122
   7.62698412  10.96177673  15.34332657   9.05166626  13.00399876]


In [11]:
print model_dir

/home/zyan3/proj/caffe_private_hdcnn/models/nin_imagenet/89clusters/89clusters_v2.0/


In [10]:
num_val = 50000
batch_size = 500

# num_val = 200
# batch_size = 50

batch_num = num_val / batch_size

topk=[1,5]
topk_accu_central_crop={}
topk_accu_10_view={}
for k in topk:
    topk_accu_central_crop[str(k)]=0
    topk_accu_10_view[str(k)]=0

pred_labels_central_crop = np.zeros((num_val),dtype=np.int32)
pred_labels_10_view = np.zeros((num_val),dtype=np.int32)

'''in the case of fine2multicoarse, need to do L1 normalization to make prediction a valid probability distribution'''
prediction_single=np.zeros((num_val, num_class), dtype=np.single)
prediction_multi=np.zeros((num_val, num_class), dtype=np.single)
gt_labels=np.zeros((num_val),dtype=np.int32)

for i in range(batch_num):
    print 'batch %d out of %d' % (i+1, batch_num)
    start, end = i*batch_size,(i+1)*batch_size
    
    test_imgs =[]
    test_img_labels=[]
    for j in range(start,end):
        key = '%s' %(val_img_names[j])
        string_data=txn.get(key)
        datum = caffe_pb2.Datum()
        datum.ParseFromString(string_data)    
        data_array = np.fromstring(datum.data, dtype=np.uint8) 
        data_array = np.float32(data_array)    
        data_array = data_array.reshape((datum.channels, datum.height, datum.width))
        data_array = np.transpose(data_array,(1,2,0))
        data_array /= 255.0
        test_imgs += [data_array]
        test_img_labels += [datum.label]
    gt_labels[start:end]=np.asarray(test_img_labels)
    
    '''single view test'''
    st_time=time.time()
    prediction = net.predict(test_imgs, oversample = False)
    prediction_single[start:end, :]=prediction
    
    pred_labels = np.zeros((batch_size, max(topk)),dtype=np.int32)
    for j in range(batch_size):
        idx=np.argsort(prediction[j,:])[::-1]
        pred_labels[j,:]=idx[:max(topk)]
    pred_labels_central_crop[start:end]=pred_labels[:,0]
        
    batch_topk_accu, batch_topk_guesses = top_k_accuracy(pred_labels, test_img_labels, topk)
    for key, value in batch_topk_accu.iteritems():
        topk_accu_central_crop[key] += value
    ep_time=time.time()-st_time
    print 'single view test elapsed time %3.2f seconds. top %d accu %4.3f top %d accuracy %4.3f' % \
    (ep_time,topk[0],batch_topk_accu[str(topk[0])], topk[-1],batch_topk_accu[str(topk[-1])])
    
    '''10-view test'''
    st_time=time.time()
    prediction = net.predict(test_imgs, oversample = True)
    prediction_multi[start:end,:]=prediction
    pred_labels = np.zeros((batch_size, max(topk)),dtype=np.int32)
    for j in range(batch_size):
        idx=np.argsort(prediction[j,:])[::-1]
        pred_labels[j,:]=idx[:max(topk)]
    pred_labels_10_view[start:end]=pred_labels[:,0]
    
    batch_topk_accu, batch_topk_guesses = top_k_accuracy(pred_labels, test_img_labels, topk)
    for key, value in batch_topk_accu.iteritems():
        topk_accu_10_view[key] += value
    ep_time=time.time()-st_time
    print '10-view test elapsed time %3.2f seconds. top %d accu %4.3f top %d accuracy %4.3f' % \
    (ep_time,topk[0],batch_topk_accu[str(topk[0])], topk[-1],batch_topk_accu[str(topk[-1])])
    
print '-----------results-----------'
for key, value in topk_accu_central_crop.iteritems():
    topk_accu_central_crop[key]/=batch_num
    print 'w/o oversample top %s accuracy: %5.4f' % (key,topk_accu_central_crop[key])
    topk_accu_10_view[key]/=batch_num
    print 'with 10-view oversample top %s accuracy: %5.4f' % (key,topk_accu_10_view[key])

'''save prediction data into file'''
net_prediction={}
net_prediction['single']=prediction_single
net_prediction['multi']=prediction_multi
net_prediction['gt_labels']=gt_labels
pickle(model_dir + 'net_predictions', net_prediction)

batch 1 out of 100
10-view test elapsed time 384.57 seconds. top 1 accu 0.660 top 5 accuracy 0.872
batch 2 out of 100
10-view test elapsed time 385.35 seconds. top 1 accu 0.614 top 5 accuracy 0.830
batch 3 out of 100
10-view test elapsed time 393.44 seconds. top 1 accu 0.652 top 5 accuracy 0.856
batch 4 out of 100
10-view test elapsed time 389.55 seconds. top 1 accu 0.604 top 5 accuracy 0.810
batch 5 out of 100
10-view test elapsed time 389.14 seconds. top 1 accu 0.656 top 5 accuracy 0.860
batch 6 out of 100
10-view test elapsed time 391.40 seconds. top 1 accu 0.652 top 5 accuracy 0.828
batch 7 out of 100
10-view test elapsed time 386.56 seconds. top 1 accu 0.658 top 5 accuracy 0.862
batch 8 out of 100
10-view test elapsed time 388.45 seconds. top 1 accu 0.646 top 5 accuracy 0.826
batch 9 out of 100
10-view test elapsed time 390.71 seconds. top 1 accu 0.610 top 5 accuracy 0.830
batch 10 out of 100
10-view test elapsed time 388.43 seconds. top 1 accu 0.668 top 5 accuracy 0.830
batch 11 

In [ ]:
print '-----------results-----------'
for key, value in topk_accu_central_crop.iteritems():
    topk_accu_central_crop[key]/=batch_num
    print 'w/o oversample top %s accuracy: %5.4f' % (key,topk_accu_central_crop[key])
    topk_accu_10_view[key]/=batch_num
    print 'with 10-view oversample top %s accuracy: %5.4f' % (key,topk_accu_10_view[key])

In [16]:
dict={}
dict['pred_labels_central_crop']=pred_labels_central_crop
dict['pred_labels_10_view']=pred_labels_10_view
pickle(model_dir + '89clusters/89clusters_v2.0/vgg_16_layer_validation_results', dict)

In [5]:
'''load prediction data from file'''
net_prediction=unpickle(model_dir+'net_predictions')
